In [209]:
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, String, Date, Numeric, DECIMAL
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv




In [210]:
# Load environment variables from .env file
load_dotenv()

MYSQL_USERNAME = os.getenv('MYSQL_USERNAME')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_PORT = os.getenv('MYSQL_PORT')
MYSQL_DB = os.getenv('STAGING_DB')

#

In [211]:
# Construct the MySQL connection string
if MYSQL_PASSWORD:
    connection_string = f"mysql+pymysql://{MYSQL_USERNAME}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
else:
    connection_string = f"mysql+pymysql://{MYSQL_USERNAME}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)
print('Connection to MySQL database established successfully.')


Connection to MySQL database established successfully.


In [212]:
query = "SELECT * FROM distressed_companies_staging"  
df = pd.read_sql(query, con=engine)

query = "SELECT * FROM us_companies_staging"  
us_data = pd.read_sql(query, con=engine)


query = "SELECT * FROM companies_staging"  
companies =  = pd.read_sql(query, con=engine)

# Extract the data from the database
query = "SELECT * FROM company_financials_staging"  # Example query to extract all data
us_companies_df = pd.read_sql(query, con=engine)

# Close the connection
engine.dispose()

# print("Data extraction completed successfully.")

In [213]:
us_companies_df.shape

(80839, 29)

In [214]:
us_companies_df.columns

Index(['company_id', 'date', 'industry', 'state', 'company_status',
       'size_category', 'age', 'company_name', 'bankrupt', 'year_founded',
       'total_revenue', 'net_sales', 'gross_profit', 'cost_of_goods_sold',
       'total_operating_expenses', 'depreciation_and_amortization', 'ebit',
       'ebitda', 'net_income', 'total_assets', 'total_liabilities', 'equity',
       'current_assets', 'total_current_liabilities', 'inventory',
       'total_receivables', 'retained_earnings', 'market_value',
       'total_long_term_debt'],
      dtype='object')

In [215]:
us_companies_df.rename(columns={
    'X1':'current_assets',
    'X2':'cost_of_goods_sold',
    'X3':'depreciation_and_amortization',
    'X4':'ebitda',
    'X5':'inventory',
    'X6':'net_income',
    'X7':'total_receivables',
    'X8':'market_value',
    'X9':'net_sales',
    'X10':'total_assets',
    'X11':'total_long_term_debt',
    'X12':'ebit',
    'X13':'gross_profit',
    'X14':'total_current_liabilities',
    'X15':'retained_earnings',
    'X16':'total_revenue',
    'X17':'total_liabilities',
    'X18':'total_operating_expenses'}, inplace = True)

In [216]:
# make a copy of the dataframe
us_companies = us_companies_df.copy()

#### standarize for consistency

In [217]:
us_companies['bankrupt'].value_counts()

bankrupt
0    68701
1    12138
Name: count, dtype: int64

In [218]:

us_companies.rename(columns={'status_label': 'bankrupt', 'company_status': 'ownership', 'year_founded': 'date_founded', 'size_category': 'size'}, inplace=True)


In [219]:
# drop year column
# us_companies.drop(columns=['year'], inplace=True)

In [220]:
us_companies['bankrupt'].value_counts()

bankrupt
0    68701
1    12138
Name: count, dtype: int64

In [221]:
# map distressd and healthy to the bankrupt column
us_companies['bankrupt'] = us_companies['bankrupt'].map({1: 'Bankrupt', 0: 'Profitable'})

In [222]:
us_companies['bankrupt'].value_counts()

bankrupt
Profitable    68701
Bankrupt      12138
Name: count, dtype: int64

### Prepare DataFrames for Dimension Table

##### Prepare Dim_State

In [223]:
Dim_State = pd.DataFrame({
    'state_name': us_companies['state'].unique()
})
Dim_State.index.name = 'state_id'
# Dim_State.head
# Dim_State.shape

##### Prepare Dim_Industry

In [224]:


Dim_Industry = pd.DataFrame({
    'industry_name': us_companies['industry'].unique()
})
Dim_Industry.index.name = 'industry_id'
# Dim_Industry.shape
# Dim_Industry.head


In [225]:
Dim_Industry

,industry_name
industry_id,
0,Retail
1,Finance
2,Manufacturing
3,Healthcare
4,Technology



##### Prepare Dim_Date

In [226]:
unique_dates = pd.to_datetime(us_companies['date']).dropna().unique()

# Create Dim_Date using unique_dates
Dim_Date = pd.DataFrame({
    'date': unique_dates,
    'year': [date.year for date in unique_dates],
    'quarter': [date.quarter for date in unique_dates],
    'month': [date.month for date in unique_dates],
    'day': [date.day for date in unique_dates]
})

# Set an index name for SQL table consistency
Dim_Date.index.name = 'date_id'

# Dim_Date.head()
# Dim_Date.shape

##### Prepare Dim_Company

In [227]:
# Include 'industry' and 'state' in the initial DataFrame creation
# Dim_Company = us_companies[['company_id', 'company_name']].drop_duplicates().copy()

In [228]:

# Include 'industry' and 'state' in the initial DataFrame creation
Dim_Company = us_companies.drop_duplicates(subset=['company_id']).copy()

Dim_Company = Dim_Company[['company_id', 'company_name', 'ownership', 'date', 'industry', 'state', 'bankrupt', 'size', 'age']].drop_duplicates().copy()

# # Ensure that the Dim_Industry and Dim_State DataFrames have been reset and set indices as needed
if 'industry_id' not in Dim_Industry.columns:
    Dim_Industry.reset_index(inplace=True)
if 'state_id' not in Dim_State.columns:
    Dim_State.reset_index(inplace=True)

# Create mappings for 'industry_id' and 'state_id'
industry_map = Dim_Industry.set_index('industry_name')['industry_id'].to_dict()
state_map = Dim_State.set_index('state_name')['state_id'].to_dict()

# Map 'industry_id' and 'state_id' using the dictionaries
Dim_Company['industry_id'] = Dim_Company['industry'].map(industry_map)
Dim_Company['state_id'] = Dim_Company['state'].map(state_map)

# Now it's safe to drop the 'industry' and 'state' columns
Dim_Company.drop(['industry', 'state'], axis=1, inplace=True)

# # Set 'company_id' as the index name if it's meant to be the primary key
Dim_Company.index.name = 'id'

# Dim_Company.sample(5)
# Dim_Company.shape

In [229]:
Dim_Company.shape

(3000, 9)

##### Prepare Fact_Financial_Performance 

In [230]:
us_companies.columns

Index(['company_id', 'date', 'industry', 'state', 'ownership', 'size', 'age',
       'company_name', 'bankrupt', 'date_founded', 'total_revenue',
       'net_sales', 'gross_profit', 'cost_of_goods_sold',
       'total_operating_expenses', 'depreciation_and_amortization', 'ebit',
       'ebitda', 'net_income', 'total_assets', 'total_liabilities', 'equity',
       'current_assets', 'total_current_liabilities', 'inventory',
       'total_receivables', 'retained_earnings', 'market_value',
       'total_long_term_debt'],
      dtype='object')

In [231]:
# Ensure that 'date_id' and 'company_id' are correctly mapped in Fact_Financial_Performance
Fact_Financial_Performance = us_companies.copy()

# Ensure the date format is consistent
Fact_Financial_Performance['date'] = pd.to_datetime(Fact_Financial_Performance['date'], errors='coerce')

# Map 'date_id' by ensuring the data types match
Dim_Date.reset_index(inplace=True)  # Reset index to access 'date_id'
Fact_Financial_Performance['date_id'] = Fact_Financial_Performance['date'].map(
    Dim_Date.set_index('date')['date_id']
)



# Check if the mappings worked correctly (No missing values)
if Fact_Financial_Performance[['company_id', 'date_id']].isnull().any().any():
    print("Warning: Missing values in 'company_id' or 'date_id' mapping!")

# Final structure
Fact_Financial_Performance.sample(5)


,company_id,date,industry,state,ownership,size,age,company_name,bankrupt,date_founded,...,total_liabilities,equity,current_assets,total_current_liabilities,inventory,total_receivables,retained_earnings,market_value,total_long_term_debt,date_id
12808,475,2009-03-31,Retail,North Carolina,Private,Medium,10,"Garner, Castro and May",Profitable,1987,...,4.950160e+06,4.896115e+06,5.624871e+06,1.596765e+06,1.769949e+06,2.390313e+06,1.840713e+07,5.395908e+06,3.353395e+06,76
40826,1508,2017-03-31,Finance,North Carolina,Private,Medium,8,Foster Ltd,Profitable,2003,...,4.570891e+07,3.119918e+07,5.094247e+07,2.483696e+07,5.463347e+07,5.455902e+07,3.830012e+08,3.431261e+07,2.087194e+07,14
77710,2883,2016-06-30,Finance,Ohio,Public,Large,14,"Kelley, Davis and Jones",Profitable,1984,...,1.368490e+08,3.171196e+08,3.603348e+08,6.107303e+07,7.576025e+07,5.963429e+07,9.027831e+08,4.251990e+08,7.577602e+07,11
45488,1684,2014-06-30,Healthcare,Ohio,Private,Medium,14,Wolf-Salazar,Profitable,1986,...,1.265720e+07,1.536400e+07,1.989105e+07,6.868087e+06,5.390685e+06,6.593137e+06,4.593099e+07,1.799897e+07,5.789113e+06,3
41823,1546,2011-03-31,Manufacturing,Illinois,Public,Medium,5,"Roth, Murray and Ruiz",Profitable,1979,...,4.949793e+07,4.053855e+07,6.657190e+07,2.969387e+07,4.577748e+06,3.940812e+06,-4.107274e+06,5.112704e+07,1.980406e+07,84


In [232]:
Fact_Financial_Performance.sample(2)


,company_id,date,industry,state,ownership,size,age,company_name,bankrupt,date_founded,...,total_liabilities,equity,current_assets,total_current_liabilities,inventory,total_receivables,retained_earnings,market_value,total_long_term_debt,date_id
60636,2246,2014-09-30,Manufacturing,Georgia,Private,Small,3,"Rogers, Miller and Young",Profitable,2001,...,4.191496e+06,3.090925e+06,4.951701e+06,1.330103e+06,3.864926e+06,3.519676e+06,2.470852e+07,4.386372e+06,2.861393e+06,4
4804,177,2013-03-31,Finance,Georgia,Public,Medium,11,Nelson-Mcgrath,Profitable,1983,...,3.886911e+07,4.510774e+07,5.349719e+07,1.835345e+07,1.268180e+07,8.797833e+06,5.515661e+07,6.158754e+07,2.051566e+07,53


In [233]:
# Calculating the KPIs
Fact_Financial_Performance['current_ratio'] = Fact_Financial_Performance['current_assets'] / Fact_Financial_Performance['total_current_liabilities']
Fact_Financial_Performance['quick_ratio'] = (Fact_Financial_Performance['current_assets'] - Fact_Financial_Performance['inventory']) / Fact_Financial_Performance['total_current_liabilities']
Fact_Financial_Performance['debt_to_equity_ratio'] = Fact_Financial_Performance['total_liabilities'] / (Fact_Financial_Performance['total_assets'] - Fact_Financial_Performance['total_liabilities'])
Fact_Financial_Performance['gross_margin'] = Fact_Financial_Performance['gross_profit'] / Fact_Financial_Performance['net_sales']
Fact_Financial_Performance['operating_margin'] = Fact_Financial_Performance['ebit'] / Fact_Financial_Performance['total_revenue']
Fact_Financial_Performance['ebitda_margin'] = Fact_Financial_Performance['ebitda'] / Fact_Financial_Performance['total_revenue']
Fact_Financial_Performance['net_profit_margin'] = Fact_Financial_Performance['net_income'] / Fact_Financial_Performance['total_revenue']
Fact_Financial_Performance['return_on_assets'] = Fact_Financial_Performance['net_income'] / Fact_Financial_Performance['total_assets']
Fact_Financial_Performance['return_on_equity'] = Fact_Financial_Performance['net_income'] / (Fact_Financial_Performance['total_assets'] - Fact_Financial_Performance['total_liabilities'])
Fact_Financial_Performance['price_to_earnings'] = Fact_Financial_Performance['market_value'] / Fact_Financial_Performance['net_income']
Fact_Financial_Performance['price_to_sales'] = Fact_Financial_Performance['market_value'] / Fact_Financial_Performance['net_sales']
Fact_Financial_Performance['equity_multiplier'] = Fact_Financial_Performance['total_assets'] / (Fact_Financial_Performance['total_assets'] - Fact_Financial_Performance['total_liabilities'])
Fact_Financial_Performance['retention_ratio'] = Fact_Financial_Performance['retained_earnings'] / Fact_Financial_Performance['net_income']

# Sort by company_id and date_id to ensure calculations are made in order
Fact_Financial_Performance.sort_values(['company_id', 'date_id'], inplace=True)

# Calculate Revenue Growth: percentage change in total revenue
Fact_Financial_Performance['revenue_growth'] = Fact_Financial_Performance.groupby('company_id')['total_revenue'].pct_change() * 100

# Calculate Profit Growth: percentage change in net income
Fact_Financial_Performance['profit_growth'] = Fact_Financial_Performance.groupby('company_id')['net_income'].pct_change() * 100


# Drop unnecessary columns
Fact_Financial_Performance.drop(['state', 'industry', 'ownership', 'company_name', 'date', 'date_founded', 'bankrupt', 'age'], axis=1, inplace=True)
Fact_Financial_Performance['record_id'] = range(1, len(Fact_Financial_Performance) + 1)


# Display the updated DataFrame to ensure that revenue_growth and profit_growth are calculated
Fact_Financial_Performance.sample(5)

,company_id,size,total_revenue,net_sales,gross_profit,cost_of_goods_sold,total_operating_expenses,depreciation_and_amortization,ebit,ebitda,...,net_profit_margin,return_on_assets,return_on_equity,price_to_earnings,price_to_sales,equity_multiplier,retention_ratio,revenue_growth,profit_growth,record_id
71740,2661,Medium,3.617607e+07,3.617607e+07,2.826200e+07,7.914066e+06,1.912328e+07,1.704743e+06,7.433979e+06,9.138722e+06,...,0.143626,0.123158,0.256316,5.832405,0.837687,2.081199,13.423458,-2.301008,-79.296007,71741
48683,1804,Medium,3.324716e+07,3.324716e+07,2.357190e+07,9.675259e+06,9.931007e+06,7.779447e+05,1.286294e+07,1.364089e+07,...,0.286910,0.443296,0.856888,1.257805,0.360877,1.932992,6.179429,12.555730,22.132507,48684
21069,781,Small,1.750000e+07,1.750000e+07,6.914688e+06,1.058531e+07,1.314799e+07,7.022128e+05,-6.935512e+06,0.000000e+00,...,-0.410829,-1.101451,-2.330634,-0.479757,0.197098,2.115968,0.749613,-2.349694,-17.726106,21070
71560,2655,Medium,1.203233e+08,1.203233e+08,7.940896e+07,4.091435e+07,2.265470e+07,5.151304e+06,5.160296e+07,5.675426e+07,...,0.325875,0.456740,0.672444,1.591251,0.518548,1.472270,3.122891,41.106145,-171.200686,71566
79726,2961,Small,5.658447e+06,5.658447e+06,3.592008e+06,2.066439e+06,2.051381e+06,2.521094e+05,1.288517e+06,1.540627e+06,...,0.162621,0.274128,0.462677,2.252906,0.366369,1.687813,4.902615,7.839942,-39.270315,79727


In [234]:
#Find rows where either EBITDA or Total Revenue is negative
negative_ebitda = us_companies[us_companies['ebitda'] < 0]
negative_revenue = us_companies[us_companies['total_revenue'] < 0]

print(f"Number of negative EBITDA values: {len(negative_ebitda)}")
print(f"Number of negative Total Revenue values: {len(negative_revenue)}")

Number of negative EBITDA values: 0
Number of negative Total Revenue values: 0


In [235]:
Dim_Company['company_id'].duplicated().sum()

np.int64(0)

In [236]:
# # find out the inf and -inf values
# Fact_Financial_Performance.replace([np.inf], max_value, inplace=True)
# Fact_Financial_Performance.replace([-np.inf], min_value, inplace=True)



In [237]:
Fact_Financial_Performance.columns

Index(['company_id', 'size', 'total_revenue', 'net_sales', 'gross_profit',
       'cost_of_goods_sold', 'total_operating_expenses',
       'depreciation_and_amortization', 'ebit', 'ebitda', 'net_income',
       'total_assets', 'total_liabilities', 'equity', 'current_assets',
       'total_current_liabilities', 'inventory', 'total_receivables',
       'retained_earnings', 'market_value', 'total_long_term_debt', 'date_id',
       'current_ratio', 'quick_ratio', 'debt_to_equity_ratio', 'gross_margin',
       'operating_margin', 'ebitda_margin', 'net_profit_margin',
       'return_on_assets', 'return_on_equity', 'price_to_earnings',
       'price_to_sales', 'equity_multiplier', 'retention_ratio',
       'revenue_growth', 'profit_growth', 'record_id'],
      dtype='object')

In [238]:
Fact_Financial_Performance.replace([np.inf, -np.inf], np.nan, inplace=True)  # Temporarily replace inf and -inf with NaN
Fact_Financial_Performance.dropna(inplace=True)  # Drop rows with NaN, which includes rows that had inf or -inf



In [239]:
# MYSQL_DB = os.getenv('DATA_WAREHOUSE_DB')
MYSQL_DB = 'frr_dw'
# Construct the MySQL connection string
if MYSQL_PASSWORD:
    connection_string = f"mysql+pymysql://{MYSQL_USERNAME}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
else:
    connection_string = f"mysql+pymysql://{MYSQL_USERNAME}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)
print('Connection to MySQL database established successfully.')


Connection to MySQL database established successfully.


In [240]:
Dim_Company[Dim_Company['industry_id'] == 3].shape

(448, 9)

In [241]:
# Load Dim_State DataFrame into Dim_State table
Dim_State.to_sql('dim_state', con=engine, if_exists='append', index=False)

# Load Dim_Industry DataFrame into Dim_Industry table
Dim_Industry.to_sql('dim_industry', con=engine, if_exists='append', index=False)

# Load Dim_Date DataFrame into Dim_Date table
Dim_Date.to_sql('dim_date', con=engine, if_exists='append', index=False)

# Load Dim_Company DataFrame into Dim_Company table
Dim_Company.to_sql('dim_company', con=engine, if_exists='append', index=False)

# Load Fact_Financial_Performance DataFrame into Fact_Financial_Performance table
Fact_Financial_Performance.to_sql('fact_financial_performance', con=engine, if_exists='append', index=False)

print("Data loaded into MySQL tables successfully.")
engine.dispose()


Data loaded into MySQL tables successfully.


In [242]:
# # list the columns of all datasets
# print('Columns of distressed companies dataset:', distressed_companies.columns)
# print('Columns of US companies dataset:', us_companies.columns)
# print('Columns of healthy companies dataset:', healthy_companies.columns)

In [243]:
# how many records has net income between 10 and 100
us_companies[(us_companies['net_income'] >= 10) & (us_companies['net_income'] <= 100000)].shape[0]

2675

'X1':'current_assets',
    'X2':'cost_of_goods_sold',
    'X3':'depreciation_and_amortization',
    'X4':'ebitda',
    'X5':'inventory',
    'X6':'net_income',
    'X7':'total_receivables',
    'X8':'market_value',
    'X9':'net_sales',
    'X10':'total_assets',
    'X11':'total_long_term_debt',
    'X12':'ebit',
    'X13':'gross_profit',
    'X14':'total_current_liabilities',
    'X15':'retained_earnings',
    'X16':'total_revenue',
    'X17':'total_liabilities',
    'X18':'total_operating_expenses